In [1]:
import pandas as pd
import json


# Get edge_cases where our model cannot predict correctly
edge_cases = pd.read_csv("/kaggle/input/llm-daigt-find-edge-case/edge_cases.csv")
edge_cases

,id,text,prediction,generated
0,33895,First impressions are a crucial aspect of our...,0.410707,1
1,39951,"As an eighth-grade student, I possess a talent...",0.413961,1
2,26725,"Ummm... hey there! So, umm... Winston Churchi...",0.467152,1
3,35647,"""When you are doing something wrong and someo...",0.356944,1
4,27976,I believe that working 10 hours a day is more...,0.450531,1
...,...,...,...,...
165,26115,Cell phones have become a hot topic when it co...,0.490097,1
166,29732,Honesty is a virtue that is often associated ...,0.331009,1
167,36240,The advantages of limiting car usage are becom...,0.473156,1
168,39819,Drivers Should Not Use Cell Phones in Any Capa...,0.419902,1


In [2]:
edge_cases["generated"].unique()

array([1])

In [3]:
edge_cases["prediction"].min(), edge_cases["prediction"].mean() , edge_cases["prediction"].median(),edge_cases["prediction"].max()

(0.0, 0.40149828802332427, 0.4246961745658604, 0.4985209873471804)

In [4]:
pd.DataFrame(pd.cut(edge_cases['prediction'], [0.0, 0.1, 0.2, 0.3, 0.4, 0.5], include_lowest=False).value_counts())

,count
prediction,
"(0.4, 0.5]",109
"(0.3, 0.4]",43
"(0.2, 0.3]",12
"(0.1, 0.2]",2
"(0.0, 0.1]",0


In [5]:
with open("/kaggle/input/llm-daigt-find-edge-case/metrics.json") as f:
    metrics = json.load(f)
print(metrics)

{'AUC': 0.9985770475470891}


In [6]:
import sys
import gc

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

In [7]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
# org_train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')

train = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv", sep=',')

In [8]:
train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)

In [9]:
LOWERCASE = False
VOCAB_SIZE = 14000000

In [10]:
# Creating Byte-Pair Encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
# Adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)
# Creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])
def train_corp_iter(): 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/44868 [00:00<?, ?it/s]

In [11]:
tokenized_texts_test[1]

['ĠBbb', 'Ġccc', 'Ġddd', '.']

In [12]:
def dummy(text):
    return text

In [13]:
def dummy(text):
    return text

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, analyzer = 'word',
    tokenizer = dummy,
    preprocessor = dummy,
    token_pattern = None, strip_accents='unicode')

vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode'
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)

del vectorizer
gc.collect()

{'ĠAaa Ġbbb Ġccc': 0, 'Ġbbb Ġccc .': 6, 'ĠAaa Ġbbb Ġccc .': 1, 'ĠBbb Ġccc Ġddd': 2, 'Ġccc Ġddd .': 7, 'ĠBbb Ġccc Ġddd .': 3, 'ĠCCC Ġddd Ġeee': 4, 'Ġddd Ġeee .': 8, 'ĠCCC Ġddd Ġeee .': 5}


23

In [14]:
y_train = train['label'].values

In [15]:
def get_model():
    from catboost import CatBoostClassifier

#     clf2 = MultinomialNB(alpha=0.01)
    clf = MultinomialNB(alpha=0.0225)
#     clf2 = MultinomialNB(alpha=0.01)
    sgd_model = SGDClassifier(max_iter=9000, tol=1e-4, loss="modified_huber", random_state=6743) 
    p6={'n_iter': 3000,'verbose': -1,'objective': 'cross_entropy','metric': 'auc',
        'learning_rate': 0.00581909898961407, 'colsample_bytree': 0.78,
        'colsample_bynode': 0.8,
#         'lambda_l1': 4.562963348932286, 
       # 'lambda_l2': 2.97485, 'min_data_in_leaf': 115, 'max_depth': 23, 'max_bin': 898
       }
    p6["random_state"] = 6743
    lgb=LGBMClassifier(**p6)
    cat=CatBoostClassifier(iterations=3000,
                           verbose=0,
                           random_seed=6543,
#                            l2_leaf_reg=6.6591278779517808,
                           learning_rate=0.005599066836106983,
                           subsample = 0.35,
                           allow_const_label=True,loss_function = 'CrossEntropy')
    weights = [0.2,0.31,0.31,0.46]
 
    ensemble = VotingClassifier(estimators=[('mnb',clf),
                                            ('sgd', sgd_model),
                                            ('lgb',lgb), 
                                            ('cat', cat)
                                           ],
                                weights=weights, voting='soft', n_jobs=-1)
    return ensemble

model = get_model()
print(model)

if len(test.text.values) <= 5:
    # if not, just sample submission
    sub.to_csv('submission.csv', index=False)
else:
    model.fit(tf_train, y_train)

    gc.collect()

    final_preds = model.predict_proba(tf_test)[:,1]
    sub['generated'] = final_preds
    sub.to_csv('submission.csv', index=False)
    sub

VotingClassifier(estimators=[('mnb', MultinomialNB(alpha=0.0225)),
                             ('sgd',
                              SGDClassifier(loss='modified_huber',
                                            max_iter=9000, random_state=6743,
                                            tol=0.0001)),
                             ('lgb',
                              LGBMClassifier(colsample_bynode=0.8,
                                             colsample_bytree=0.78,
                                             learning_rate=0.00581909898961407,
                                             metric='auc', n_iter=3000,
                                             objective='cross_entropy',
                                             random_state=6743, verbose=-1)),
                             ('cat',
                              <catboost.core.CatBoostClassifier object at 0x790dfb0d52d0>)],
                 n_jobs=-1, voting='soft', weights=[0.2, 0.31, 0.31, 0.46])
